# component.port

> TODO fill in description

In [ ]:
#| default_exp comp.port

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from enum import Enum
from typing import List, Dict, Callable, Any
from types import MappingProxyType
from dataclasses import dataclass, field
import re, keyword

import fbdev
from fbdev.comp.packet import Packet
from fbdev._utils import SingletonMeta, AttrContainer, StateHandler, StateView, StateCollection

In [ ]:
#|hide
from fbdev.comp.packet import Packet

In [ ]:
#|hide
show_doc(fbdev.comp.port.PortType)

---

### PortType

>      PortType (value, names=None, module=None, qualname=None, type=None,
>                start=1)

*An enumeration.*

In [ ]:
#|export
class PortType(Enum):
    INPUT = ("input", True)
    CONFIG = ("config", True)
    SIGNAL = ("signal", True)
    
    OUTPUT = ("output", False)
    MESSAGE = ("message", False)
    
    def __init__(self, label:str, is_input_port:bool):
        self._label:str = label
        self._is_input_port:bool = is_input_port
        
    @property
    def label(self) -> str: return self._label
    @property
    def is_input_port(self) -> bool: return self._is_input_port
    
    def get(self, port_type_label:str) -> PortType:
        for port_type in self:
            if port_type.label == port_type_label:
                return port_type
        raise RuntimeError(f"Port type {port_type_label} does not exist.")

In [ ]:
#|hide
show_doc(fbdev.comp.port.PortSpec)

---

### PortSpec

>      PortSpec (port_type, name, dtype=None, data_validator=None,
>                is_optional=False, default=<fbdev.utils.NO_DEFAULT object at
>                0x107ac8040>)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PortSpec:
    _NO_DEFAULT = SingletonMeta('NO_DEFAULT')
    
    def __init__(self, port_type, name, dtype=None, data_validator=None, is_optional=False, default=_NO_DEFAULT()):
        self._name:str = name
        self._port_type:PortType = port_type
        self._full_name:str = f"{port_type.label}.{name}"
        self._dtype:type = dtype
        self._data_validator:Callable[[Any], bool] = data_validator
        self._is_optional = is_optional
        self._default = default
        
        if not self.is_valid_port_name(self.name):
            raise ValueError(f"Invalid port name '{self.name}'.")
        
        if port_type == PortType.SIGNAL:
            if dtype is not None: raise RuntimeError(f"Signal port {self.name} cannot have a dtype.")
            if data_validator is not None: raise RuntimeError(f"Signal port {self.name} cannot have a data validator.")
        
        if port_type != PortType.CONFIG:
            if is_optional:
                raise RuntimeError(f"Only ports of type {PortType.CONFIG} can be optional.")
            if type(default) != PortSpec._NO_DEFAULT:
                raise RuntimeError(f"Only ports of type {PortType.CONFIG} can have a default value.")
        
        if self.is_optional and self.has_default:
            raise RuntimeError("Config port {self.name} cannot have both be optional and have a default value.")
            
    @property
    def name(self) -> str: return self._name
    @property
    def full_name(self) -> str: return self._full_name
    @property
    def port_type(self) -> PortType: return self._port_type
    @property
    def is_input_port(self) -> bool: return self._port_type.is_input_port
    @property
    def is_output_port(self) -> bool: return not self.is_input_port
    @property
    def dtype(self) -> type: return self._dtype
    @property
    def data_validator(self) -> Callable[[Any], bool]: return self._data_validator
    
    @property
    def has_dtype(self) -> bool: return self._dtype is not None
    @property
    def has_data_validator(self) -> bool: return self._data_validator is not None
    
    @property
    def is_optional(self) -> bool: return self._is_optional
    @property
    def default(self) -> Any:
        if not self.has_default: raise RuntimeError(f"Config port {self.name} does not have a default value.")
        return self._default
    @property
    def has_default(self) -> bool: return type(self._default) != PortSpec._NO_DEFAULT
    
    def __str__(self) -> str:
        return f"{self.port_type}.{self.name}"
    
    def __repr__(self) -> str:
        return str(self)

    def copy(self):
        if self.has_default:
            port_spec = PortSpec(
                self._port_type,
                self._name,
                self._dtype,
                self._data_validator,
                self._is_optional,
                self._default
            )
        else:
            port_spec = PortSpec(
                self._port_type,
                self._name,
                self._dtype,
                self._data_validator,
                self._is_optional
            )
        return port_spec
    
    @classmethod
    def is_valid_port_name(cls, name: str) -> bool:
        """
        Check if the provided string is a valid Python variable name.
        
        Parameters:
        name (str): The string to check.
        
        Returns:
        bool: True if the string is a valid Python variable name, False otherwise.
        """
        # Check if the name is a Python keyword
        if keyword.iskeyword(name):
            return False
        
        # Regular expression to match valid Python identifiers
        valid_identifier_pattern = r'^[A-Za-z_][A-Za-z0-9_]*$'
        
        # Use the regular expression to check the validity of the variable name
        if re.match(valid_identifier_pattern, name):
            return True
        else:
            return False

In [ ]:
# Example usage:
assert PortSpec.is_valid_port_name("my_var")
assert not PortSpec.is_valid_port_name("2var")
assert not PortSpec.is_valid_port_name("def")
assert not PortSpec.is_valid_port_name("my.var")
assert not PortSpec.is_valid_port_name("my.var!")

In [ ]:
#|hide
show_doc(fbdev.comp.port.PortSpecCollection)

---

### PortSpecCollection

>      PortSpecCollection (*port_specs:List[PortSpec])

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PortSpecCollection:
    def __init__(self, *port_specs:List[PortSpec]):
        self._readonly:bool = False
        self._ports: Dict[str, PortSpec] = {}
        for port_type in PortType:
            setattr(self, port_type.label, AttrContainer({}, obj_name=f"{PortSpecCollection.__name__}.{port_type.label}", dtype=PortSpec))
        for port_spec in port_specs:
            if not isinstance(port_spec, PortSpec):
                raise TypeError(f"PortSpecCollection can only contain PortSpecs. Got '{type(port_spec)}'.")
            self.add_port(port_spec)
    
    def __getitem__(self, key) -> PortSpec:
        if type(key) == tuple:
            if type(key[0]) != PortType or type(key[1]) != str or len(key) != 2:
                raise TypeError(f"Key must be a tuple of (PortType, str). Got '{key}'.")
            key = f"{key[0].label}.{key[1]}"
        if key in self._ports: return self._ports[key]
        else: raise KeyError(f"'{key}' does not exist in {self.__class__.__name__}.")
    
    def __iter__(self): return self._ports.__iter__()
    def __len__(self): return self._ports.__len__()
    def __contains__(self, key): return key in self._ports
    def as_dict(self) -> Dict[str, PortSpec]: return MappingProxyType(self._ports)
    def iter_ports(self) -> List[PortSpec]: return self._ports.values()
    
    def make_readonly(self): self._readonly = True
    
    def add_port(self, port_spec:PortSpec):
        if self._readonly: raise RuntimeError("Cannot add ports to a readonly PortSpecCollection.")
        if port_spec.full_name in self._ports: raise ValueError(f"Port name '{port_spec.name}' already exists in {self.__class__.__name__}.")
        self._ports[port_spec.full_name] = port_spec
        getattr(self, port_spec.port_type.label)._set(port_spec.name, port_spec)
    
    def remove_port(self, port_spec:PortSpec):
        if self._readonly: raise RuntimeError("Cannot remove ports from a readonly PortSpecCollection.")
        if port_spec.full_name not in self._ports: raise ValueError(f"Port name '{port_spec.name}' does not exist in {self.__class__.__name__}.")
        del self._ports[port_spec.full_name]
        getattr(self, port_spec.port_type.label)._remove(port_spec.name)
        
    def update(self, parent:PortSpecCollection):
        if self._readonly: raise RuntimeError("Cannot add ports to a readonly PortSpecCollection.")
        for port in parent._ports.values():
            self.add_port(port)
        
    def copy(self) -> PortSpecCollection:
        """Note: The copy is not readonly."""
        port_spec_collection = PortSpecCollection(
            *[port_spec.copy() for port_spec in self._ports.values()]
        )
        return port_spec_collection
        
    def __str__(self) -> str:
        lines = []
        for port_type in PortType:
            if len(getattr(self, port_type.label)) == 0: continue
            lines.append(f"{port_type.label}:")
            for port_spec in getattr(self, port_type.label).values():
                line = f"  {str(port_spec.name)}"
                if port_spec.dtype is not None: line += f":{port_spec.dtype.__name__}"
                if port_spec.has_default: line += f"={port_spec.default.__repr__()}"
                lines.append(line)
        return "\n".join(lines)
    
    def __repr__(self):
        return self.__str__()
    

In [ ]:
PortSpecCollection(
    PortSpec(PortType.INPUT,'in1'),
    PortSpec(PortType.OUTPUT,'out1', dtype=int),
    PortSpec(PortType.CONFIG,'conf1', dtype=str, default=''),
)

input:
  in1
config:
  conf1:str=''
output:
  out1:int

In [ ]:
#|hide
show_doc(fbdev.comp.port.Port)

---

### Port

>      Port (port_spec:PortSpec)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class Port:
    def __init__(self, port_spec:PortSpec):
        self._name: str = port_spec.name
        self._full_name: str = port_spec.full_name
        self._port_type: PortType = port_spec.port_type
        self._is_input_port: bool = port_spec.is_input_port
        self._dtype: type = port_spec.dtype
        self._data_validator: Callable[[Any], bool] = port_spec.data_validator
        self._packet: Packet = None
        
        self._states = StateCollection()
        self._states._add_state(StateHandler("is_blocked", False)) # If input port, it's blocked if the component is currently getting. If output port, it's blocked if the component is currently putting.
        self._states._add_state(StateHandler("put_awaiting", False))
        self._states._add_state(StateHandler("get_awaiting", False))
        
        self._packet_queue = asyncio.Queue(maxsize=1)
        self._gets_are_waiting_cond = asyncio.Condition()
        self._num_waiting_gets = 0
        self._num_waiting_puts = 0
        
        if self._is_input_port: self.get = self._get
        else: self.put = self._put
        
    @property
    def name(self) -> str: return self._name
    @property
    def full_name(self) -> str: return self._full_name
    @property
    def port_type(self) -> PortType: return self._port_type
    @property
    def dtype(self) -> type: return self._dtype
    @property
    def is_input_port(self) -> bool: return self._is_input_port
    @property
    def is_output_port(self) -> bool: return not self.is_input_port
    @property
    def data_validator(self) -> Callable[[Any], bool]: return self._data_validator
    @property
    def states(self) -> StateCollection: return self._states
        
    async def _put(self, packet:Packet):
        if not isinstance(packet, Packet): raise ValueError(f"`packet` is not of type `{Packet.__name__}`.")
        
        if not self._is_input_port: self.states._is_blocked.set(True)
        self._num_waiting_puts += 1
        self.states._put_awaiting.set(True)
        
        async with self._gets_are_waiting_cond:
            await self._gets_are_waiting_cond.wait_for(lambda: self._num_waiting_gets > 0)
            if self._packet is not None:
                raise RuntimeError(f"Critical runtime error: Port '{self.name}' is already full.")
            await self._packet_queue.put(packet)
            self._num_waiting_puts -= 1
            if self._num_waiting_puts == 0:
                self.states._put_awaiting.set(False)
                if not self._is_input_port: self.states._is_blocked.set(False)
    
    async def _get(self) -> Packet:
        if self._is_input_port: self.states._is_blocked.set(True)
        self.states._get_awaiting.set(True)
        self._num_waiting_gets += 1
        async with self._gets_are_waiting_cond: self._gets_are_waiting_cond.notify()
        packet = await self._packet_queue.get()
        self._num_waiting_gets -= 1
        async with self._gets_are_waiting_cond: self._gets_are_waiting_cond.notify()
        if self._num_waiting_gets == 0:
            self.states._get_awaiting.set(False)
            if self._is_input_port: self.states._is_blocked.set(False)
        return packet

In [ ]:
port_spec = PortSpec(PortType.INPUT, 'in1')
port = Port(port_spec)

tasks = []

for i in range(10):
    packet = Packet(f'datum #{i}')
    async def print_data():
        packet = await port.get()
        data = await packet.consume()
        print(data)
    tasks.append(asyncio.create_task(print_data()))
    
for i in range(10):
    packet = Packet(f'datum #{i}') 
    tasks.append(asyncio.create_task(port._put(packet)))
    
await asyncio.gather(*tasks);

datum #0
datum #1
datum #2
datum #3
datum #4
datum #5
datum #6
datum #7
datum #8
datum #9


In [ ]:
port_spec = PortSpec(PortType.INPUT, 'in1')
port = Port(port_spec)

async def put_packet():
    print('Putting packet')
    packet = Packet(f'data')
    await port._put(packet)
    
asyncio.create_task(put_packet())
asyncio.create_task(put_packet())
await asyncio.sleep(0)
print(f'port.states.put_awaiting={port.states.put_awaiting.get()}')
await port._get()
print(f'port.states.put_awaiting={port.states.put_awaiting.get()}')
await port._get()
print(f'port.states.put_awaiting={port.states.put_awaiting.get()}')

Putting packet
Putting packet
port.states.put_awaiting=True
port.states.put_awaiting=True
port.states.put_awaiting=False


In [ ]:
port_spec = PortSpec(PortType.INPUT, 'in1')
port = Port(port_spec)

async def get_packet():
    await port._get()
    
asyncio.create_task(get_packet())
asyncio.create_task(get_packet())
await asyncio.sleep(0)
print(f'port.states.get_awaiting={port.states.get_awaiting.get()}')
await port._put(Packet(f'data'))
print(f'port.states.get_awaiting={port.states.get_awaiting.get()}')
await port._put(Packet(f'data'))
await asyncio.sleep(0)
print(f'port.states.get_awaiting={port.states.get_awaiting.get()}')

port.states.get_awaiting=True
port.states.get_awaiting=True
port.states.get_awaiting=False


In [ ]:
#|hide
show_doc(fbdev.comp.port.PortCollection)

---

### PortCollection

>      PortCollection (port_spec_collection:PortSpecCollection)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PortCollection:
    def __init__(self, port_spec_collection:PortSpecCollection):
        self._port_spec_collection: PortSpecCollection = port_spec_collection
        self._ports: Dict[str, Port] = {}
        for port_type in PortType:
            setattr(self, port_type.label, AttrContainer({}, obj_name=f"{PortCollection.__name__}.{port_type.label}", dtype=Port))
        for port_spec in port_spec_collection.as_dict().values():
            self._ports[port_spec.full_name] = port = Port(port_spec)
            getattr(self, port_spec.port_type.label)._set(port_spec.name, port)
    
    def __getitem__(self, key) -> Port:
        if type(key) == tuple:
            if type(key[0]) != PortType or type(key[1]) != str or len(key) != 2:
                raise TypeError(f"Key must be a tuple of (PortType, str). Got '{key}'.")
            key = f"{key[0].label}.{key[1]}"
        if key in self._ports: return self._ports[key]
        else: raise KeyError(f"'{key}' does not exist in {self.__class__.__name__}.")
    
    def __iter__(self): return self._ports.__iter__()
    def __len__(self): return self._ports.__len__()
    def __contains__(self, key):
        if type(key) == tuple:
            if type(key[0]) != PortType or type(key[1]) != str or len(key) != 2:
                raise TypeError(f"Key must be a tuple of (PortType, str). Got '{key}'.")
            key = f"{key[0]}.{key[1]}"
        return key in self._ports
    def as_dict(self) -> Dict[str, Port]: return MappingProxyType(self._ports)
    

    def __str__(self): return self._port_spec_collection.__str__()
    
    def __repr__(self): return self._port_spec_collection.__repr__()

In [ ]:
port_spec_collection = PortSpecCollection(
    PortSpec(PortType.INPUT,'in1'),
    PortSpec(PortType.OUTPUT,'out1', dtype=int),
    PortSpec(PortType.CONFIG,'conf1', dtype=str, default=''),
)

PortCollection(port_spec_collection)

input:
  in1
config:
  conf1:str=''
output:
  out1:int